In [17]:
import pandas as pd
import textacy
import spacy
nlp = spacy.load('en')
from iso3166 import countries

Intermediate file manually edited. Donot lose that !!!!!
'WWFHighRisk_intermediate.xlsx'

In [18]:
def get_df():
    file = 'WWF_HighRiskCountryProfiles_Species.xlsx'
    df = pd.read_excel(file)
    return df
df = get_df()

countries = list(df['Country of Origin Risk'])

def process_country_row(sent):
    c_list = sent.replace("," , " , ")
    c_list = c_list.replace('also via', ',')
    c_list = c_list.replace('and via', ',')
    c_list = c_list.replace('all via', ',')
    c_list = c_list.replace(' all', ',')
    c_list = c_list.replace(' also', ',')
    c_list = c_list.replace(' and', ',')
    c_list = c_list.replace(' & ', ',')
    c_list = c_list.replace(' also ', ',')
    c_list = c_list.replace(' via ', ',')
    c_list = c_list.replace('-',' ')
    c_list = textacy.preprocess.normalize_whitespace(c_list)
    c_list = c_list.split('(Note')[0]
    c_list = c_list.split (',')
    
    res_list=[]
    for item in c_list:
        item = item.strip()
        if len(item) < 20 and len (item) > 0 :
            res_list.append(item)
            
    return res_list

# ---------- #
# Create a list of all countries , to clean up
# ---------- #
all_countries = []
for c_list in countries:
    if type(c_list)!= str:
        continue
    res_list = process_country_row(c_list)
    all_countries.extend(res_list)
    
all_countries = list(set(all_countries))    
print ('All countries')
print(all_countries)
print('----')

All countries
['Russia', 'Solomon Islands', 'Bolivia', 'Myanmar', 'Cameroon', 'Malaysia', 'Lao DR', 'Brazil', 'Peru', 'Indonesia', 'China', 'Panama', 'El Salvador', 'DRC', 'Papua New Guinea', 'Gabon', 'Vietnam']
----


In [19]:
#  -------------------------------------- #
#  Replace the country names!
#  Manually curated List 
#  Using iso31366 package
#  from iso3166 import countries
#  print(countries.get('us'))
#  for c in countries:
#        print (c)
#  -------------------------------------- #
ref_corr_country_names = {
    'Bolivia' : 'Bolivia, Plurinational State of',
    'Russia' : 'Russian Federation',
    'Lao DR' : "Lao People's Democratic Republic",
    'DRC' : 'Congo, Democratic Republic of the',
    'Vietnam' : "Viet Nam"
}
from iso3166 import countries
replace_dict = {}

for c in all_countries:
    if c not in ref_corr_country_names.keys():
        res = countries.get(c)
    else:
        c_name = ref_corr_country_names[c]
        res = countries.get(c_name)
        
    if res is not None:   
        replace_dict[c] = res.alpha2
    else :
        print ("error")
    


In [20]:
# Place the country names back in the Data Frame
# using replace_dict
from iso3166 import countries
for i,row in df.iterrows():
    sent = row['Country of Origin Risk']
    p1 = None 
    p2 = None
    if type(sent) == str :    
        # split on via
        sent = sent.replace(","," , ")
        parts = sent.split(' via ')
        if len(parts) > 1 :
            p1 = process_country_row(parts[0])
            p2 = process_country_row(parts[1])
        else :
            p1 = process_country_row(parts[0])
        
        origins = None
        conduit = None
        if p1 is not None:
            origins = []
        
        for p in p1 :
            if p in ref_corr_country_names.keys():
                p = ref_corr_country_names[p]
            origins.extend(countries.get(p).alpha2)
            
        origins = ';'.join(origins)
        if p2 is not None:
            conduit = []
            for p in p2 :
                if p in ref_corr_country_names.keys():
                    p = ref_corr_country_names[p]
                conduit.append(countries.get(p).alpha2)
            conduit = ';'.join(conduit)
    df.loc[i,'origin'] = origins
    df.loc[i,'conduit'] = conduit

In [21]:
# filter out Columns needed
df = df[['Taxonomic Genus','Species, if applicable','Common Names','origin','conduit']]
df = df.rename(columns ={
    'Taxonomic Genus' : 'genus',
    'Species, if applicable': 'species',
    'Common Names' : 'common_name'
})


In [23]:
# df.to_excel('WWFHighRisk_intermediate.xlsx')
df = pd.read_excel('WWFHighRisk_intermediate_edited.xlsx')
                
for i,row in df.iterrows():
    cn = row['common_name']
    if type(cn) != str:
        continue
    cn = cn.split(',')    
    res = []
    for c in cn:
        c = c.strip()
        c = c.lower()
        c = textacy.preprocess.normalize_whitespace(c)
        res.append(c)
    res = ';'.join(res)
    df.loc[i,'common_name'] = res
df.to_csv('WWF_HighRisk_final.csv')

